<a href="https://colab.research.google.com/github/LeonVillanueva/CoLab/blob/master/Google_CoLab_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -q tensorflow==2.0.0-beta1

In [0]:
%%capture
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [0]:
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Dropout, GlobalMaxPooling2D, GaussianNoise, BatchNormalization, MaxPooling2D, SimpleRNN, GRU, LSTM, GlobalMaxPooling1D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

In [0]:
from scipy import stats
import math
import seaborn as sns
import re
from nltk.stem import WordNetLemmatizer
import re

> *https://www.gutenberg.org/files/4357/4357-h/4357-h.htm*

In [0]:
text = ['She flew down the stairs, found the basket and returned with it to the attic.',
        'Then she sat down before the brass-studded box and began trying one key after another in the curious old lock.',
        'Some were too large, but most were too small.']

max_vocab_size = 20000

tokenizer = Tokenizer (num_words = max_vocab_size)
tokenizer.fit_on_texts (text)
sequences = tokenizer.texts_to_sequences (text)

In [0]:
data = pad_sequences (sequences, padding='post')

> *https://www.kaggle.com/jrobischon/wikipedia-movie-plots*

In [80]:
!unzip movp.zip

Archive:  movp.zip
replace wiki_movie_plots_deduped.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [0]:
df = pd.read_csv ('wiki_movie_plots_deduped.csv')

In [82]:
df.columns.values

array(['Release Year', 'Title', 'Origin/Ethnicity', 'Director', 'Cast',
       'Genre', 'Wiki Page', 'Plot'], dtype=object)

In [0]:
df = df[(df['Origin/Ethnicity'] == 'American') & (df['Genre'] != 'unknown')]

In [0]:
genres = df['Genre']

In [0]:
top_genres = genres.value_counts()[:10].index.tolist()

In [0]:
series = df [df['Genre'] == top_genres[0]].sample(300)

In [0]:
for i in range (1,len(top_genres)):
  series_b = df [df['Genre'] == top_genres[i]].sample(300)
  series = pd.concat((series, series_b), axis=0)

In [0]:
final = series[['Genre','Plot']]

In [0]:
lemmatizer = WordNetLemmatizer()

In [0]:
from nltk.corpus import stopwords
import nltk

In [91]:
nltk.download ('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

> *https://chrisalbon.com/machine_learning/preprocessing_text/remove_stop_words/*

> *https://h1ros.github.io/posts/stemming-words-and-sentences/*

> *https://www.datacamp.com/community/tutorials/stemming-lemmatization-python*

In [0]:
def simplify (x):
  stop_words = stopwords.words ('english')
  words = re.sub(r'[^\w\s]','',x).lower().split(' ')
  words = [lemmatizer.lemmatize (word) for word in words if word not in stop_words]
  return words

In [0]:
txt = final.iloc[0]['Plot']

In [101]:
len (simplify (txt))

965

In [102]:
final['Plot'].apply (simplify)

6993     [s, berwind, rusty, old, ship, chartered, line...
8015     [laura, reynolds, taylor, freespirited, unwed,...
1099     [princess, ling, moy, life, next, door, petrie...
284      [described, film, magazine5, clive, standing, ...
10847    [around, christmas, united, state, supreme, co...
                               ...                        
6961     [film, begin, brownie, troop, visiting, doll, ...
7299     [pathologist, dr, warren, chapin, price, disco...
8738     [21st, century, sexual, intercourse, reproduct...
15412    [1928, solitary, mountaineer, encounter, glowi...
5595     [story, take, place, new, york, city, postapoc...
Name: Plot, Length: 3000, dtype: object